<a href="https://colab.research.google.com/github/victorgil777/CUVD/blob/main/MODELOS_DE_SEGURIDAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




**Pronóstico del rendimiento estudiantil en cursos universitarios virtuales a distancia (CUVD).** 

A continuación se presenta el código que debe guardar en su cuenta de Google Drive.

In [ ]:
# Primer paso importar la librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import warnings
import tensorflow
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, Normalizer, OrdinalEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical
warnings.filterwarnings("ignore")

A continuación cargue la base de datos de los estudiantes, recuerde que debe nombrar el archivo "ESTUDIANTES.csv".

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ESTUDIANTES.csv to ESTUDIANTES.csv


In [ ]:
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('ESTUDIANTES', name)


ESTUDIANTES ESTUDIANTES.csv


In [ ]:
import pandas as pd
df = pd.read_csv('ESTUDIANTES.csv')


In [ ]:
df = df.sample(frac=1)

In [ ]:
print(df.shape)

(1340, 6)


In [ ]:
df.head()

,Resultado_final,Edad,Estrato,Genero,Creditos_aprobados,Total_clics
362,0,21,3,1,112,132
1283,1,18,1,1,99,72
1132,0,19,3,1,70,45
803,0,23,4,1,115,135
867,0,21,3,0,75,67


In [ ]:
# Rango del Quartil

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
inter_quartile_range = Q3 - Q1

In [ ]:
# Rango de los cuartiles e intercuartiles

df_out = df[~((df < (Q1 - (1.5 * inter_quartile_range))) | (df > (Q3 + (1.5 * inter_quartile_range)))).any(axis=1)]
df.shape, df_out.shape

((1340, 6), (1332, 6))

In [ ]:
# Definr la etiqueta de salida y lvas variables predictoras
X = df_out.drop(columns=['Resultado_final'])
y = df_out['Resultado_final']

In [ ]:
# Definir el conjunto de entrenamiento y el conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# we used 30% test data
# check the size before beginning
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((932, 5), (400, 5), (932,), (400,))

In [ ]:
# Función par calcular la validación cruzada
def tn(y_true, y_pred):
    return confusion_matrix(y_true, y_pred)[0, 0]

In [ ]:
def fp(y_true, y_pred):
    return confusion_matrix(y_true, y_pred)[0, 1]

In [ ]:
def fn(y_true, y_pred):
    return confusion_matrix(y_true, y_pred)[1, 0]

In [ ]:
def tp(y_true, y_pred):
    return confusion_matrix(y_true, y_pred)[1, 1]

In [ ]:
def acc(y_true, y_pred):
    return accuracy(y_true, y_pred)

In [ ]:
# Accuracy
def accuracy(y_true, y_pred):
    cnf_matrix = confusion_matrix(y_true, y_pred)
    N = sum(map(sum, cnf_matrix))
    tp = cnf_matrix[1, 1]
    tn = cnf_matrix[0, 0]
    return round((tp + tn) / N, 2)

In [ ]:
# Propósito de la validación cruzada
scoring = {'accuracy': make_scorer(metrics.accuracy_score), 'prec': 'precision'}
scoring = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
           'fp': make_scorer(fp), 'fn': make_scorer(fn),
           'acc': make_scorer(acc)}

In [ ]:
def print_result(result):
    print("True Positive: ", result['test_tp'])
    print("True Negative: ", result['test_tn'])
    print("False Negative: ", result['test_fn'])
    print("False Positive: ", result['test_fp'])
    print("Accuracy: ", result['test_acc'])

In [ ]:
# Lista alamcenada de las acc y rcc de las salidas de cada modelo
acc = []
roc = []

In [ ]:
# Primer Modelo - Naive Bayes
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
ac = accuracy_score(y_test, y_pred)
acc.append(ac)
ac_train = accuracy_score(y_train, y_pred_train)
rc = roc_auc_score(y_test, y_pred)
roc.append(rc)
print("Naive Bayes : ")
print("Training Set Accuracy : ", ac_train)
print("Test Set Accuracy {0} ROC {1}".format(ac, rc))

Naive Bayes : 
Training Set Accuracy :  0.8047210300429185
Test Set Accuracy 0.7575 ROC 0.7606416066826623


In [ ]:
# Validación cruzada
result = cross_validate(model, X_train, y_train, scoring=scoring, cv=10)
print_result(result)

True Positive:  [26 23 25 25 23 24 29 23 31 30]
True Negative:  [50 52 45 50 49 48 44 56 42 50]
False Negative:  [7 9 7 7 9 8 3 9 1 2]
False Positive:  [11 10 16 11 12 13 17  5 19 11]
Accuracy:  [0.81 0.8  0.75 0.81 0.77 0.77 0.78 0.85 0.78 0.86]


In [ ]:
# Segundo Modelo - Regresión logística
LR = LogisticRegression()
LR.fit(X_train, y_train)
LR.score(X_train, y_train)
y_pred = LR.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
# Accuracy
ac = accuracy_score(y_test, y_pred)
acc.append(ac)
ac_train = accuracy_score(y_train, y_pred_train)
# Code for ROC_AUC curve
rc = roc_auc_score(y_test, y_pred)
roc.append(rc)
print("****************************************************")
print("Logistic Regression : ")
print("Training Set Accuracy : ", ac_train)
print("Test Set Accuracy {0} ROC {1}".format(ac, rc))

****************************************************
Logistic Regression : 
Training Set Accuracy :  0.8047210300429185
Test Set Accuracy 0.8175 ROC 0.7802511922746528


In [ ]:
# La validación cruzada muestra la precisión del modelo
result = cross_validate(LR, X_train, y_train, scoring=scoring, cv=10)
print_result(result)

True Positive:  [19 18 21 19 20 18 27 21 27 23]
True Negative:  [54 58 53 53 56 55 50 57 55 55]
False Negative:  [14 14 11 13 12 14  5 11  5  9]
False Positive:  [ 7  4  8  8  5  6 11  4  6  6]
Accuracy:  [0.78 0.81 0.8  0.77 0.82 0.78 0.83 0.84 0.88 0.84]


In [ ]:
# Tercer Modelo - Support Vector Machine
model = SVC(gamma='auto', kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
ac = accuracy_score(y_test, y_pred)
acc.append(ac)
ac_train = accuracy_score(y_train, y_pred_train)
rc = roc_auc_score(y_test, y_pred)
roc.append(rc)
print("****************************************************")
print("Support Vector Machine : ")
print("Training Set Accuracy : ", ac_train)
print("Test Set Accuracy {0} ROC {1}".format(ac, rc))

****************************************************
Support Vector Machine : 
Training Set Accuracy :  0.842274678111588
Test Set Accuracy 0.8725 ROC 0.8243579489913802


In [ ]:
# Validación cruzada
result = cross_validate(model, X_train, y_train, scoring=scoring, cv=10)
print_result(result)

True Positive:  [21 16 18 21 20 18 26 22 28 23]
True Negative:  [56 59 54 57 56 57 55 57 60 57]
False Negative:  [12 16 14 11 12 14  6 10  4  9]
False Positive:  [5 3 7 4 5 4 6 4 1 4]
Accuracy:  [0.82 0.8  0.77 0.84 0.82 0.81 0.87 0.85 0.95 0.86]


In [ ]:
# Cuarto Modelo - KNN
model = KNeighborsClassifier(n_neighbors=9, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
ac = accuracy_score(y_test, y_pred)
acc.append(ac)
ac_train = accuracy_score(y_train, y_pred_train)
rc = roc_auc_score(y_test, y_pred)
roc.append(rc)
print("****************************************************")
print("K-Nearest Neighbors : ")
print("Training Set Accuracy : ", ac_train)
print("Accuracy {0} ROC {1}".format(ac, rc))

****************************************************
K-Nearest Neighbors : 
Training Set Accuracy :  0.871244635193133
Accuracy 0.845 ROC 0.8116650374714891


In [ ]:
# Validación cruzada
result = cross_validate(model, X_train, y_train, scoring=scoring, cv=10)
print_result(result)

True Positive:  [23 19 19 25 22 21 22 21 29 21]
True Negative:  [58 60 55 58 56 58 54 58 54 56]
False Negative:  [10 13 13  7 10 11 10 11  3 11]
False Positive:  [3 2 6 3 5 3 7 3 7 5]
Accuracy:  [0.86 0.84 0.8  0.89 0.84 0.85 0.82 0.85 0.89 0.83]


In [ ]:
# Quinto Modelo - Random Forest
model = RandomForestClassifier(n_estimators=20, max_depth=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
ac = accuracy_score(y_test, y_pred)
acc.append(ac)
ac_train = accuracy_score(y_train, y_pred_train)
rc = roc_auc_score(y_test, y_pred)
roc.append(rc)
print("****************************************************")
print("Random Forest : ")
print("Training Set Accuracy : ", ac_train)
print("Accuracy {0} ROC {1}".format(ac, rc))

****************************************************
Random Forest : 
Training Set Accuracy :  0.9753218884120172
Accuracy 0.865 ROC 0.8283420717438313


In [ ]:
# Validación cruzada
result = cross_validate(model, X_train, y_train, scoring=scoring, cv=10)
print_result(result)

True Positive:  [26 21 19 24 23 19 24 22 29 24]
True Negative:  [54 57 55 53 52 54 51 57 54 55]
False Negative:  [ 7 11 13  8  9 13  8 10  3  8]
False Positive:  [ 7  5  6  8  9  7 10  4  7  6]
Accuracy:  [0.85 0.83 0.8  0.83 0.81 0.78 0.81 0.85 0.89 0.85]


In [ ]:
# Sexto Modelo - Árboles de decisión
model = DecisionTreeClassifier(criterion = 'entropy', random_state=0)
model.fit(X_train,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
ac = accuracy_score(y_test, y_pred)
acc.append(ac)
ac_train = accuracy_score(y_train, y_pred_train)
rc = roc_auc_score(y_test, y_pred)
roc.append(rc)
print("****************************************************")
print("Decision Tree : ")
print("Training Set Accuracy : ", ac_train)
print("Accuracy {0} ROC {1}".format(ac, rc))

****************************************************
Decision Tree : 
Training Set Accuracy :  0.9753218884120172
Accuracy 0.865 ROC 0.8283420717438313


In [ ]:
# Validacion cruzada
result = cross_validate(model, X_train, y_train, scoring=scoring, cv=10)
print_result(result)

True Positive:  [27 21 22 23 26 19 25 22 28 23]
True Negative:  [56 52 54 53 48 53 45 52 46 53]
False Negative:  [ 6 11 10  9  6 13  7 10  4  9]
False Positive:  [ 5 10  7  8 13  8 16  9 15  8]
Accuracy:  [0.88 0.78 0.82 0.82 0.8  0.77 0.75 0.8  0.8  0.82]


In [ ]:
# Septimo Modelo - Red Neuronal Artificial
scaler = MinMaxScaler()
df_transform = scaler.fit_transform(df)

In [ ]:
# Fixing random seed for reproducibility
seed = 85
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
  # create model
  model = Sequential()
  model.add(Dense(5, input_dim=5, activation='relu'))
  #model.add(Dropout(0.2))  # <-- added these later to see effect on score
  model.add(Dense(4, activation='sigmoid'))
  #model.add(Dropout(0.2))  # <-- added these later to see effect on score, omitted from baseline.
  model.add(Dense(1, activation='sigmoid'))

  # compile model
  adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  print(model.summary())
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32,
          verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 24        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 59
Trainable params: 59
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
30/30 [==============================] - 1s 19ms/step - loss: 0.5825 - accuracy: 0.7270 - val_loss: 0.4675 - val_accuracy: 0.8575
Epoch 2/100
30/30 [==============================] - 0s 3ms/step - loss: 0.4874 - accuracy: 0.8258 - val_loss: 0.4238 - val_accuracy: 0.8675
Epoch 3/100
30/30 [==============================] - 0s 3ms/step - loss: 0.4458 - accuracy: 0.8460 - v

In [ ]:
print(f"Accuracy: {accuracy_score(np.round(model.predict(X_test)),y_test)}")

Accuracy: 0.87


In [ ]:
y_pred=model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[264,  15],
       [ 37,  84]])

In [ ]:
# Encontrar los mejores parámetros
# Function to create model, required for KerasClassifier:
def gridsearch_create_model():
  # create model
  model = Sequential()
  model.add(Dense(5, input_dim=5, activation='relu'))
  model.add(Dense(4, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))

  # compile model
  adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  return model

  # create model
model2 = KerasClassifier(build_fn=gridsearch_create_model, verbose=0)

  # define the grid search parameters
param_grid = {'batch_size': [32, 64, 128, 256, 512],
                'epochs': [100]}

                # see "Results, with tuning:" cell below for version changes
                # note: original batch_size: [10, 20, 40, 60, 80, 100],

In [ ]:
# Create grid search
grid = GridSearchCV(estimator=model2, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print(f"Means: {mean}, Stdev: {stdev} with: {params}")

Best: 0.8348053574562073 using {'batch_size': 128, 'epochs': 100}
Means: 0.8100454211235046, Stdev: 0.08424034760142785 with: [{'batch_size': 32, 'epochs': 100}, {'batch_size': 64, 'epochs': 100}, {'batch_size': 128, 'epochs': 100}, {'batch_size': 256, 'epochs': 100}, {'batch_size': 512, 'epochs': 100}]
Means: 0.8110976934432983, Stdev: 0.07694546329281274 with: [{'batch_size': 32, 'epochs': 100}, {'batch_size': 64, 'epochs': 100}, {'batch_size': 128, 'epochs': 100}, {'batch_size': 256, 'epochs': 100}, {'batch_size': 512, 'epochs': 100}]
Means: 0.8348053574562073, Stdev: 0.04052673574360078 with: [{'batch_size': 32, 'epochs': 100}, {'batch_size': 64, 'epochs': 100}, {'batch_size': 128, 'epochs': 100}, {'batch_size': 256, 'epochs': 100}, {'batch_size': 512, 'epochs': 100}]
Means: 0.6621988415718079, Stdev: 0.18101564663191708 with: [{'batch_size': 32, 'epochs': 100}, {'batch_size': 64, 'epochs': 100}, {'batch_size': 128, 'epochs': 100}, {'batch_size': 256, 'epochs': 100}, {'batch_size':